In [49]:
import pandas as pd
import utils
import plotly.graph_objects as go

In [50]:
pair = "EUR_USD"
granularity = "H1"

In [52]:
df = pd.read_pickle(utils.get_his_data_filename(pair, granularity))

In [53]:
non_cols = ['time', 'volume']

In [54]:
non_cols

['time', 'volume']

In [55]:
mod_cols = [x for x in df.columns if x not in non_cols]

In [56]:
df[mod_cols] = df[mod_cols].apply(pd.to_numeric)

In [57]:
df_plot = df.iloc[-100:].copy()

In [58]:
df_plot.shape

(100, 14)

In [60]:
fig = go.Figure()
fig.add_trace(go.Candlestick(
  x=df_plot.time, open=df_plot.mid_o, high=df_plot.mid_h, low=df_plot.mid_l, close=df_plot.mid_c,
  line=dict(width=1), opacity=1,
  increasing_fillcolor="#24A06B",
  decreasing_fillcolor="#CC2E3C",
  increasing_line_color="#2EC886",
  decreasing_line_color="#FF3A4C"
))
fig.update_layout(width=1000,height=300,
    margin=dict(l=10, r=10, b=10, t=10),
    font=dict(size=10, color='#e1e1e1'),
    paper_bgcolor="#1e1e1e",
    plot_bgcolor="#1e1e1e")
fig.update_xaxes(
  gridcolor='#1f292f',
  showgrid=True,fixedrange=True,rangeslider=dict(visible=False)
)
fig.update_yaxes(
  gridcolor="#1f292f",
  showgrid=True
)
fig.show()